# 組み込み型の不快入れ子にはせずクラスを作成する

In [16]:
# 名前が前もって分かっていない学生集団の成績を記録するとする
class SimpleGradebook:
  def __init__(self):
    self._grades = {}

  def add_student(self, name):
    self._grades[name] = []

  def report_grade(self, name, score):
    self._grades[name].append(score)

  def average_grade(self, name):
    grades = self._grades[name]
    return sum(grades) / len(grades)

In [17]:
# クラスの使い方は簡単
book = SimpleGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isaac Newton', 90)
book.report_grade('Isaac Newton', 95)
book.report_grade('Isaac Newton', 85)

print(book.average_grade('Isaac Newton'))

90.0


In [18]:
# だが、辞書とその関連の組み込み型はあまりにも使いやすいので、
# 拡張しすぎて脆弱なコードを書いてしまう危険がある

# 科目ごとに成績のリストを管理するようにしたいとする
from collections import defaultdict

class BySubjectGradebook:
  def __init__(self):
    self._grades = {}                            # 外側の dict

  def add_student(self, name):
    self._grades[name] = defaultdict(list)       # 内側の dict

  def report_grade(self, name, subject, grade):
    by_subject = self._grades[name]
    grade_list = by_subject[subject]
    grade_list.append(grade)

  def average_grade(self, name):
    by_subject = self._grades[name]
    total, count = 0, 0
    for grades in by_subject.values():
      total += sum(grades)
      count += len(grades)
    return total / count

In [19]:
# まだ単純
book = BySubjectGradebook()
book.add_student('Albert Einstein')
book.report_grade('Albert Einstein', 'Math', 75)
book.report_grade('Albert Einstein', 'Math', 65)
book.report_grade('Albert Einstein', 'Gym', 90)
book.report_grade('Albert Einstein', 'Gym', 95)
print(book.average_grade('Albert Einstein'))

81.25


In [20]:
# 各テスト（期末、中間、抜き打ち）に重みを付けるとする
class WeightedGradebook:
  def __init__(self):
    self._grades = {}

  def add_student(self, name):
    self._grades[name] = defaultdict(list)

  def report_grade(self, name, subject, score, weight):
    by_subject = self._grades[name]
    grade_list = by_subject[subject]
    grade_list.append((score, weight))

  # ループの中にループがあり、見づらい
  def average_grade(self, name):
    by_subject = self._grades[name]

    score_sum, score_count = 0, 0
    for subject, scores in by_subject.items():
      subject_avg, total_weight = 0, 0
      for score, weight in scores:
        subject_avg += score * weight
        total_weight += weight

      score_sum += subject_avg / total_weight
      score_count += 1
    return score_sum / score_count

In [21]:
# 使い方も明瞭ではない
# 位置引数の数値が何を意味しているかわかりずらい
book = WeightedGradebook()
book.add_student('Albert Einstein')
book.report_grade('Albert Einstein', 'Math', 75, 0.05)
book.report_grade('Albert Einstein', 'Math', 65, 0.15)
book.report_grade('Albert Einstein', 'Math', 70, 0.80)
book.report_grade('Albert Einstein', 'Gym', 100, 0.40)
book.report_grade('Albert Einstein', 'Gym', 85, 0.60)
print(book.average_grade('Albert Einstein'))

80.25


辞書を要素として含む辞書は、コードが他のプログラマにとって読みにくくなる

### クラスへのリファクタリング

タプルを次々に長くしていくことは、辞書で層を深くしていくのと同じようなものである

2要素のタプルより長くしていることが分かったら、すぐに別の方法を考えるべきである

組み込みモジュール collections の namedtuple 型は、まさに櫃お湯なことをしてくれる

※ namedtuple は多くの場合に有用であるが、弊害の方が多くなる場合もある

- デフォルト引数を指定できない
- 6個以上の属性を扱うなら、組み込みモジュール dataclasses が良い

In [28]:
from collections import namedtuple

Grade = namedtuple('Grade', ('score', 'weight'))

In [29]:
class Subject:
  def __init__(self):
    self._grades = []

  def report_grade(self, score, weight):
    self._grades.append(Grade(score, weight))

  def average_grade(self):
    total, total_weight = 0, 0
    for grade in self._grades:
      total += grade.score * grade.weight
      total_weight += grade.weight
    return total / total_weight

In [23]:
class Student:
  def __init__(self):
    self._subjects = defaultdict(Subject)

  def get_subject(self, name):
    return self._subjects[name]

  def average_grade(self):
    total, count = 0, 0
    for subject in self._subjects.values():
      total += subject.average_grade()
      count += 1
    return total / count

In [24]:
class Gradebook:
  def __init__(self):
    self._students = defaultdict(Student)

  def get_student(self, name):
    return self._students[name]

In [31]:
book = Gradebook()
albert = book.get_student('Albert Einstein')
math = albert.get_subject('Math')
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)
gym = albert.get_subject('Gym')
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)
print(albert.average_grade())

80.25
